## Sampling from the new Datacube

In [16]:
# Import required libraries
import xarray as xr
from pathlib import Path
import numpy as np
import datetime
import pandas as pd
import cfgrib
import os
import shutil
from datetime import datetime
import matplotlib.pyplot as plt

In [17]:
final_cube = xr.open_dataset(Path.home() / 'hdd1/diogenis/observatory/dwd_meteo.nc')

In [18]:
final_cube

<xarray.Dataset>
Dimensions:         (time: 171, y: 983, x: 1253)
Coordinates:
  * time            (time) datetime64[ns] 2021-07-14T12:00:00 ... 2021-12-31T...
  * x               (x) float64 18.7 18.71 18.72 18.73 ... 28.88 28.89 28.9
  * y               (y) float64 42.3 42.29 42.28 42.27 ... 34.32 34.31 34.3
Data variables: (12/15)
    max_t2m         (time, y, x) float32 ...
    min_t2m         (time, y, x) float32 ...
    avg_t2m         (time, y, x) float32 ...
    max_tp          (time, y, x) float32 ...
    min_tp          (time, y, x) float32 ...
    avg_tp          (time, y, x) float32 ...
    ...              ...
    max_v10         (time, y, x) float32 ...
    min_v10         (time, y, x) float32 ...
    avg_v10         (time, y, x) float32 ...
    max_wind_speed  (time, y, x) float32 ...
    min_wind_speed  (time, y, x) float32 ...
    avg_wind_speed  (time, y, x) float32 ...

In [19]:
def copy_files(path, label):
    if label == 'positives':
        destination_folder = Path.home() / 'hdd1/diogenis/observatory/positives'
    else:
        destination_folder = Path.home() / 'hdd1/diogenis/observatory/negatives'
        
    #For Every file
    for file_name in os.listdir(path):
        #Get the date from the file name
        file_date = file_name.split('_')[0][:4] + '/' + file_name.split('_')[0][4:6] + '/' + file_name.split('_')[0][6:]
        
        #Convert from string to datetime
        file_date = datetime.strptime(file_date, '%Y/%m/%d')
        
        #If the year is 2021
        if file_date.year == 2021:
            
            #Check for the month and the day(We start from 23 couse we want to have to 10 past days)
            if (file_date.month ==7 and file_date.day >=23) or (file_date.month >=8 and file_date.month <= 12):
                
                #We don't care about the mode.npy files
                if file_name.split('_')[3] == 'static.npy' or file_name.split('_')[3] == 'dynamic.npy' or file_name.split('_')[4] == 'vec.npy':
                    source = path / file_name
                    destination = destination_folder / file_name
                    if os.path.isfile(source):
                        shutil.copy(source, destination)

In [7]:
path = Path.home() / 'hdd1/iprapas/uc3/datasets_v4/npy/temporal/positives'
copy_files(path, 'positives')

In [8]:
path = Path.home() / 'hdd1/iprapas/uc3/datasets_v5/npy/temporal/negatives_clc'
copy_files(path, 'negatives')

In [20]:
def change_npy_files(path):
    #For every file
    for file_name in os.listdir(path):
        
        #Only if the file is dynamic.npy
        if file_name.split('_')[3] == 'dynamic.npy':
            
            #Get the date
            file_date = file_name.split('_')[0][:4] + '/' + file_name.split('_')[0][4:6] + '/' + file_name.split('_')[0][6:]
            file_date = datetime.strptime(file_date, '%Y/%m/%d')
            file_date = np.datetime64(file_date)
            
            #Add 12 hours so it matches the dimension in our datacube
            file_date = file_date + np.timedelta64(12, 'h')
            
            #Get the lat and lon
            x_index = file_name.split('_')[1]
            x_index = int(x_index)
            y_index = file_name.split('_')[2]
            y_index = int(y_index)

            #Open the file
            test = np.load(path / file_name)
            
            #For every day in the file
            for i in range(10):
                
                #Get the appropriate date
                tmp = file_date - np.timedelta64(9-i,'D')
                
                #Get the values from the datacube
                dwd_max_u10 = final_cube.sel(time = tmp).isel(x = x_index, y = y_index).max_u10.values
                dwd_max_v10 = final_cube.sel(time = tmp).isel(x = x_index, y = y_index).max_v10.values
                dwd_max_t2m = final_cube.sel(time = tmp).isel(x = x_index, y = y_index).max_t2m.values
                dwd_max_tp = final_cube.sel(time = tmp).isel(x = x_index, y = y_index).max_tp.values
            
                dwd_min_u10 = final_cube.sel(time = tmp).isel(x = x_index, y = y_index).min_u10.values
                dwd_min_v10 = final_cube.sel(time = tmp).isel(x = x_index, y = y_index).min_v10.values
                dwd_min_t2m = final_cube.sel(time = tmp).isel(x = x_index, y = y_index).min_t2m.values
                dwd_min_tp = final_cube.sel(time = tmp).isel(x = x_index, y = y_index).min_tp.values
            
                dwd_avg_u10 = final_cube.sel(time = tmp).isel(x = x_index, y = y_index).avg_u10.values
                dwd_avg_v10 = final_cube.sel(time = tmp).isel(x = x_index, y = y_index).avg_v10.values
                dwd_avg_t2m = final_cube.sel(time = tmp).isel(x = x_index, y = y_index).avg_t2m.values
                dwd_avg_tp = final_cube.sel(time = tmp).isel(x = x_index, y = y_index).avg_tp.values
            
                dwd_max_wind_speed = final_cube.sel(time = tmp).isel(x = x_index, y = y_index).max_wind_speed.values
            
                #Change the values in the file
                test[i][7] = dwd_max_u10
                test[i][8] = dwd_max_v10
                test[i][10] = dwd_max_t2m
                test[i][12] = dwd_max_tp
                test[i][13] = dwd_min_u10
                test[i][14] = dwd_min_v10
                test[i][16] = dwd_min_t2m
                test[i][18] = dwd_min_tp
                test[i][19] = dwd_avg_u10
                test[i][20] = dwd_avg_v10
                test[i][22] = dwd_avg_t2m
                test[i][24] = dwd_avg_tp
                test[i][30] = dwd_max_wind_speed
            
            #Save the new values to the file
            np.save(path / file_name, test)

In [21]:
path = Path.home() / 'hdd1/diogenis/observatory/positives'
change_npy_files(path)

In [22]:
path = Path.home() / 'hdd1/diogenis/observatory/negatives'
change_npy_files(path)